<a href="https://colab.research.google.com/github/fazlur7512/Deterministic-Vision-Transformer-MNIST/blob/main/Deterministic_ViT_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    LayerNormalization,
)
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# For multiple devices (GPUs: 4, 5, 6, 7)
# os.environ["CUDA_VISIBLE_DEVICES"] = "1,4,5,6,7"
# import imageio
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import math
import time, sys
import pickle
import timeit
from scipy.interpolate import make_interp_spline, BSpline
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from keras.optimizers import Adam
import keras.backend as K
import pandas as pd
#import wandb
#os.environ["WANDB_API_KEY"] = "key_code"

import numpy as np
#!pip install tensorflow_addons
import tensorflow as tf
from tensorflow import keras
import math
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from keras.optimizers import Adam
import keras.backend as K
import pandas as pd
plt.ioff()
mnist = tf.keras.datasets.mnist
# update_progress() : Displays or updates a console progress bar
## Accepts a float between 0 and 1. Any int will be converted to a float.
## A value under 0 represents a 'halt'.
## A value at 1 or bigger represents 100%
def update_progress(progress):
    barLength = 10  # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength * progress))
    text = "\rPercent: [{0}] {1}% {2}".format("#" * block + "-" * (barLength - block), progress * 100, status)
    sys.stdout.write(text)
    sys.stdout.flush()


class Patches(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size,  1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1,1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


class MultiHeadSelfAttention(tf.keras.layers.Layer):
  def __init__(self, embed_dim, num_heads=8):
    super(MultiHeadSelfAttention, self).__init__()
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    if embed_dim % num_heads != 0:
      raise ValueError(
        f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
      )
    self.projection_dim = embed_dim // num_heads
    self.query_dense = DDense(embed_dim)
    self.key_dense = DDense(embed_dim)
    self.value_dense = DDense(embed_dim)
    self.combine_heads = DDense(embed_dim)

  def attention(self, query, key, value):
    score = tf.matmul(query, key, transpose_b=True)
    dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
    scaled_score = score / tf.math.sqrt(dim_key)
    weights = tf.nn.softmax(scaled_score, axis=-1)
    output = tf.matmul(weights, value)
    return output, weights

  def separate_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, inputs):
    batch_size = tf.shape(inputs)[0]
    query = self.query_dense(inputs)
    key = self.key_dense(inputs)
    value = self.value_dense(inputs)
    query = self.separate_heads(query, batch_size)
    key = self.separate_heads(key, batch_size)
    value = self.separate_heads(value, batch_size)

    attention, weights = self.attention(query, key, value)
    attention = tf.transpose(attention, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
    output = self.combine_heads(concat_attention)
    return output



class LayerNorm(tf.keras.layers.Layer):
    def __init__(self, eps=1e-6, **kwargs):
      self.eps = eps
      super(LayerNorm, self).__init__(**kwargs)

    def build(self, input_shape):
      self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                   initializer=tf.keras.initializers.Ones(), trainable=True)
      self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                  initializer=tf.keras.initializers.Zeros(), trainable=True)
      super(LayerNorm, self).build(input_shape)

    def call(self, x):
      mean = K.mean(x, axis=-1, keepdims=True)
      std = K.std(x, axis=-1, keepdims=True)
      return self.gamma * (x - mean) / (std + self.eps) + self.beta

    def compute_output_shape(self, input_shape):
      return input_shape

    


class DDDense(keras.layers.Layer):
  def __init__(self, units):
    super(DDDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(name='w', shape=(input_shape[1] * input_shape[2] * input_shape[-1], self.units),
                             initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05, seed=None),
                             trainable=True,
                             )

  def call(self, input_in):
    batch_size = input_in.shape[0]
    #flatt = tf.reshape(input_in, [batch_size, -1])  # shape=[batch_size, im_size*im_size*num_channel]
    out = tf.matmul(input_in, self.w)
    return out

class Dsoftmax(keras.layers.Layer):
    def __init__(self):
      super(Dsoftmax, self).__init__()

    def call(self, input_in):
      out = tf.nn.softmax(input_in)
      return out

class DDense(keras.layers.Layer):
    def __init__(self, units=32):
        '''
        Initialize the instance attributes
        '''
        super(DDense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        '''
        Create the state of the layer (weights)
        '''
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel',
                             initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                             trainable=True)
        
        # initialize bias
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias',
                             initial_value=b_init(shape=(self.units,), dtype='float32'),
                             trainable=True)
        
    def call(self, inputs):
        '''
        Defines the computation from inputs to outputs
        '''
        return tf.matmul(inputs, self.w) + self.b


class DDropout(keras.layers.Layer):
  def __init__(self, drop_prop):
    super(DDropout, self).__init__()
    self.drop_prop = drop_prop

  def call(self, input_in, Training=True):
    if Training:
      out = tf.nn.dropout(input_in, rate=self.drop_prop)
    else:
      out = input_in
    return out

class DGeLU(keras.layers.Layer):
    def __init__(self):
        super(DGeLU, self).__init__()
    def call(self, input_in):
        out = tf.nn.gelu(input_in)
        return out

class MLP(tf.keras.layers.Layer):
    def __init__(self, hidden_features, out_features, dropout_rate=0.1):
        super(MLP, self).__init__()
        self.dense1 = DDense(hidden_features)
        self.dense2 = DDense(out_features)
        self.dropout =DDropout(dropout_rate)
        self.elu_1 = DGeLU()
    def call(self, x):
        x = self.dense1(x)
        x = self.elu_1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        y = self.dropout(x)
        return y


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.mlp = MLP(mlp_dim*2,mlp_dim,dropout)
        #self.elu_1 = DGeLU()
        self.layernorm1 = LayerNorm(eps=1e-6)
        self.layernorm2 = LayerNorm(eps=1e-6)
        self.dropout1 = DDropout(dropout)
        self.dropout2 = DDropout(dropout)

    def call(self, inputs, training):
        inputs_norm = self.layernorm1(inputs)
        attn_output = self.att(inputs_norm)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = attn_output + inputs

        out1_norm = self.layernorm2(out1)
        mlp_output = self.mlp(out1_norm)
        mlp_output = self.dropout2(mlp_output, training=training)
        return mlp_output + out1


class Deterministic_ViT(tf.keras.Model):
  def __init__(
        self,
        image_size,
        patch_size,
        num_layers,
        num_classes,
        d_model,
        num_heads,
        mlp_dim,
        channels=3,
        dropout=0.1,
    ):
      super(Deterministic_ViT, self).__init__()
      num_patches = (image_size // patch_size) ** 2
      self.patch_dim = channels * patch_size ** 2

      self.patch_size = patch_size
      self.d_model = d_model
      self.num_layers = num_layers

      self.rescale = Rescaling(1.0 / 255)
      self.pos_emb = self.add_weight(
        "pos_emb", shape=(1, num_patches + 1, d_model)
        )
      self.class_emb = self.add_weight("class_emb", shape=(1, 1, d_model))
      self.patch_proj = DDense(d_model)
      self.enc_layers = [
        TransformerBlock(d_model, num_heads, mlp_dim, dropout)
        for _ in range(num_layers)
        ]
        #self.mlp_head = tf.keras.Sequential(
        #[
        #    LayerNorm(eps=1e-6),
        #    DDense(mlp_dim, activation= DGeLU),
        #    DDropout(dropout),
        #    DDense(num_classes),
        #]
      self.mlp_head =MLP(mlp_dim, num_classes)
  def extract_patches(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1,self.patch_size, self.patch_size, 1],
            strides=[1,self.patch_size, self.patch_size, 1],
            rates=[1,1, 1, 1],
            padding="VALID",
        )
        patches = tf.reshape(patches, [batch_size, -1, self.patch_dim])
        return patches

  def call(self, x, training):
        batch_size = tf.shape(x)[0]
        x = self.rescale(x)
        patches = self.extract_patches(x)
        x = self.patch_proj(patches)

        class_emb = tf.broadcast_to(
        self.class_emb, [batch_size, 1, self.d_model]
        )
        x = tf.concat([class_emb, x], axis=1)
        x = x + self.pos_emb

        for layer in self.enc_layers:
            x = layer(x, training)

    # First (class token) is used for classification
        x = self.mlp_head(x[:, 0])
        return x




def main_function(image_size=28,patch_size=7,num_layers=2,num_classes=10,d_model=64,num_heads=2,mlp_dim=64,channels=1,
                  dropout=0.1,
                  batch_size=50, epochs=10, lr=0.001, lr_end = 0.0001,
                  Targeted=False,
                Training=True, continue_training=False, saved_model_epochs=10):
    #PATH = './saved_models/cnn_epoch_{}/'.format(epochs)
    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    trans_model = Deterministic_ViT(image_size=image_size, patch_size=patch_size, num_layers=num_layers,
                                  num_classes=num_classes, d_model= d_model, num_heads=num_heads,mlp_dim = mlp_dim,
                                  channels= channels)

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    tr_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

    num_train_steps = epochs * int(x_train.shape[0] / batch_size)
    #    step = min(step, decay_steps)
    #    ((initial_learning_rate - end_learning_rate) * (1 - step / decay_steps) ^ (power) ) + end_learning_rate
    learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=lr,
                                                                     decay_steps=num_train_steps,
                                                                     end_learning_rate=lr_end, power=2.)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)  # , clipnorm=1.0)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    

    @tf.function  # Make it fast.


    def train_on_batch(x, y):
        with tf.GradientTape() as tape:
            trans_model.trainable = True
            out = trans_model(x, training=True)
            loss = loss_fn(y, out)
            gradients = tape.gradient(loss, trans_model.trainable_weights)

            #  gradients = [(tf.where(tf.math.is_nan(grad), tf.constant(1.0e-5, shape=grad.shape), grad)) for grad in gradients]
        #  gradients = [(tf.where(tf.math.is_inf(grad), tf.constant(1.0e-5, shape=grad.shape), grad)) for grad in gradients]
        optimizer.apply_gradients(zip(gradients, trans_model.trainable_weights))
        
        return loss, out

    @tf.function
    def validation_on_batch(x, y):
        trans_model.trainable = False
        out = trans_model(x, training=False)
        total_vloss = loss_fn(y, out)
        return total_vloss, out

    @tf.function
    def test_on_batch(x, y):
        trans_model.trainable = False
        out = trans_model(x, training=False)
        return out
    if Training: 
       # wandb.init(entity = "dimah", project="DCNN_Cifar10_11layers_epochs_{}_lr_{}_latest".format(epochs, lr)) 
        
        if continue_training:
            saved_model_path = './saved_models/cnn_epoch_{}/'.format(saved_model_epochs)
            trans_model.load_weights(saved_model_path + 'Deterministic_cnn_model')
        
        train_acc = np.zeros(epochs)
        valid_acc = np.zeros(epochs)        
        train_err = np.zeros(epochs)
        valid_err = np.zeros(epochs)                      
        start = timeit.default_timer()
        for epoch in range(epochs):
            print('Epoch: ', epoch + 1, '/', epochs)            
            tr_no_steps = 0
            va_no_steps = 0
            # -------------Training--------------------
            acc_training = np.zeros(int(x_train.shape[0] / (batch_size)))
            err_training = np.zeros(int(x_train.shape[0] / (batch_size)))            
            for step, (x, y) in enumerate(tr_dataset):
                update_progress(step / int(x_train.shape[0] / (batch_size)))
                loss, out = train_on_batch(x, y)             
                err_training[tr_no_steps] = loss.numpy()              
                corr = tf.equal(tf.math.argmax(out, axis=-1), tf.math.argmax(y, axis=-1))
                accuracy = tf.reduce_mean(tf.cast(corr, tf.float32))                  
                acc_training[tr_no_steps] = accuracy.numpy()                                               
                tr_no_steps += 1                
              
            train_acc[epoch] = np.mean(np.amax(acc_training))
            train_err[epoch] = np.mean(np.amin(err_training))
            print('Training Acc  ', train_acc[epoch])
            print('Training error', train_err[epoch])       
            # ---------------Validation----------------------  
            acc_validation = np.zeros(int(x_test.shape[0] / (batch_size)))
            err_validation = np.zeros(int(x_test.shape[0] / (batch_size)))                     
            for step, (x, y) in enumerate(val_dataset):
                update_progress(step / int(x_test.shape[0] / (batch_size)))
                total_vloss, out = validation_on_batch(x, y)                   
                err_validation[va_no_steps] = total_vloss.numpy()              
                corr = tf.equal(tf.math.argmax(out, axis=-1), tf.math.argmax(y, axis=-1))
                va_accuracy = tf.reduce_mean(tf.cast(corr, tf.float32))                 
                acc_validation[va_no_steps] = va_accuracy.numpy()                
                va_no_steps += 1               
            
            valid_acc[epoch] = np.mean(np.amax(acc_validation))
            valid_err[epoch] = np.mean(np.amin(err_validation))           
            stop = timeit.default_timer() 
            #cnn_model.save_weights(PATH + 'Deterministic_cnn_model')                   
##            wandb.log({"Training Loss":  train_err[epoch],                        
##                       "Training Accuracy": train_acc[epoch],                                             
##                        "Validation Loss": valid_err[epoch],                        
##                        "Validation Accuracy": valid_acc[epoch],                       
##                        'epoch': epoch
##                       })             
            print('Total Training Time: ', stop - start)
            print(' Training Acc   ', train_acc[epoch])            
            print(' Validation Acc ', valid_acc[epoch])            
            print('------------------------------------')
            print('Training error   ', train_err[epoch])            
            print('Validation error', valid_err[epoch])                    
            # -----------------End Training--------------------------                               
           
       
if __name__ == '__main__':
    main_function()





Epoch:  1 / 10
Percent: [##########] 99.91666666666667% Training Acc   0.7400000095367432
Training error 0.785493016242981
Percent: [##########] 99.5% Total Training Time:  64.728695539
 Training Acc    0.7400000095367432
 Validation Acc  0.7200000286102295
------------------------------------
Training error    0.785493016242981
Validation error 0.7721806168556213
Epoch:  2 / 10
Percent: [##########] 99.91666666666667% Training Acc   0.8999999761581421
Training error 0.35108134150505066
Percent: [##########] 99.5% Total Training Time:  129.2266023799998
 Training Acc    0.8999999761581421
 Validation Acc  0.8799999952316284
------------------------------------
Training error    0.35108134150505066
Validation error 0.42985403537750244
Epoch:  3 / 10
Percent: [##########] 99.91666666666667% Training Acc   0.9599999785423279
Training error 0.18761028349399567
Percent: [##########] 99.5% Total Training Time:  192.42880427799992
 Training Acc    0.9599999785423279
 Validation Acc  0.9200000